# AI Agents Crash Course - Part 2 - Custom Tools

## Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Uncomment if you are not using devcontainers and want to set up a local environment
# 
# # Step 1: Create and activate a virtual environment
# #
# %python3 -m venv venv
# %source venv/bin/activate
# 
# # Step 2: Install required Python modules
# #
# %pip install -r requirements.txt
# 
# # Step 3: Verify installation
# #
# %pip list

## Pre-requisites

- You will need an API key from here: https://www.exchangerate-api.com/

## Load Required Python Modules and Libraries

### Import Essential Libraries

This code block imports all the necessary libraries for building custom tools with CrewAI:

- **`dotenv`**: Loads environment variables from `.env` files
- **`os`**: Provides access to operating system interface for environment variables
- **`requests`**: HTTP library for making API calls to external services
- **`typing.Type`**: Type hints for better code documentation and IDE support
- **`IPython.display.Markdown`**: Renders markdown output in Jupyter notebooks
- **`crewai`**: Core CrewAI components (LLM, Agent, Task, Crew)
- **`crewai.tools.BaseTool`**: Base class for creating custom tools
- **`pydantic`**: Data validation and schema definition library

In [ ]:
from dotenv import load_dotenv
import os
import requests
from typing import Type
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

## [Optional] Enable litellm debug logging

### Enable Debug Logging (Troubleshooting)

This optional code enables detailed debug logging from the `litellm` library, which is useful for:

- **Diagnosing API connection issues** with LLM providers
- **Viewing detailed request/response logs** for troubleshooting
- **Understanding token usage and rate limiting**

⚠️ **Note**: Once enabled, you'll need to restart the Jupyter kernel to disable debug logging.

In [ ]:
# Uncomment in order to enable litellm debugging for better error diagnostics
# NOTE: You will have to restart the jupyter kernel to disable debug logging once it has been enabled.
#
# import litellm
# litellm._turn_on_debug()

## Load Environment Variables and Configure LLM

This block loads environment variables from the `.env` file, including the OpenAI API Key, which is required to authenticate with OpenAI's services. It then configures the `LLM` object to use OpenAI's GPT-4 model. Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [ ]:
# Load environment variables from .env file
# Note: In devcontainer, variables are already loaded by dotenv feature,
# but load_dotenv() is safe and won't override existing environment variables
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
# 
# api_key = os.getenv('OPENAI_API_KEY')
# if not api_key:
#     raise ValueError("OPENAI_API_KEY is not set in the .env file")

# Uncomment the code block below to use ollama
# 
OLLAMA_API_BASE = os.getenv('OLLAMA_API_BASE')
if not OLLAMA_API_BASE:
    raise ValueError("OLLAMA_API_BASE is not set in the .env file")

## Configure LLM

Configures the `LLM` object to use OpenAI's GPT-4 model. 

Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [ ]:
# llm = LLM(
#     model="gpt-4o",  # Specify the OpenAI model you want to use
#     api_key=api_key
# )

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
llm = LLM(
    # model="ollama/llama3:latest",
    # model="ollama/llama3.2:1b",
    # model="ollama/deepseek-r1:latest",
    # model="ollama/gemma3:latest",
    model="ollama/gemma3n:latest",
    base_url=OLLAMA_API_BASE
)

## Verify LLM Configuration

### Universal LLM Connection Test

#### Universal LLM Connection Testing Function

This comprehensive function validates your LLM configuration across multiple providers:

- **Configuration Analysis**: Inspects LLM settings (model, base URL, API key status)
- **Provider Detection**: Automatically identifies the LLM provider (OpenAI, Ollama, etc.)
- **Connection Testing**: Validates server availability and model accessibility
- **Functionality Test**: Performs an actual LLM call to ensure everything works
- **Troubleshooting**: Provides specific error messages and resolution tips

🔧 **Purpose**: Prevents issues downstream by validating the entire LLM pipeline upfront.

In [ ]:
import requests
import os

def test_llm_connection():
    """Test LLM connection regardless of provider"""
    
    print("=== LLM Configuration Analysis ===")
    
    # Analyze LLM configuration
    model_name = getattr(llm, 'model', 'Unknown')
    base_url = getattr(llm, 'base_url', None)
    api_key_set = bool(getattr(llm, 'api_key', None))
    
    print(f"Model: {model_name}")
    print(f"Base URL: {base_url if base_url else 'Default (provider-specific)'}")
    print(f"API Key Set: {'Yes' if api_key_set else 'No'}")
    
    # Determine provider type
    provider = "unknown"
    if "ollama" in model_name.lower():
        provider = "ollama"
    elif "gpt" in model_name.lower() or "openai" in model_name.lower():
        provider = "openai"
    elif "claude" in model_name.lower() or "anthropic" in model_name.lower():
        provider = "anthropic"
    elif "gemini" in model_name.lower() or "google" in model_name.lower():
        provider = "google"
    
    print(f"Detected Provider: {provider}")
    
    # Provider-specific connection tests
    print(f"\n=== {provider.title()} Connection Test ===")
    
    if provider == "ollama" and base_url:
        try:
            # Test Ollama server availability
            test_url = f"{base_url}/api/tags"
            response = requests.get(test_url, timeout=5)
            print(f"Ollama Server Status: {response.status_code}")
            if response.status_code == 200:
                models = response.json().get('models', [])
                print(f"Available Models: {len(models)} found")
                # Check if our specific model is available
                model_available = any(model_name.replace('ollama/', '') in str(model) for model in models)
                print(f"Target Model Available: {'Yes' if model_available else 'No'}")
            else:
                print(f"Ollama server responded with status: {response.status_code}")
        except Exception as e:
            print(f"❌ Ollama server connection failed: {e}")
    
    elif provider == "openai":
        print("OpenAI connection test (API key validation happens during LLM call)")
        api_key_env = os.getenv('OPENAI_API_KEY')
        print(f"OPENAI_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    elif provider == "anthropic":
        print("Anthropic connection test")
        api_key_env = os.getenv('ANTHROPIC_API_KEY')
        print(f"ANTHROPIC_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    elif provider == "google":
        print("Google AI connection test")
        api_key_env = os.getenv('GOOGLE_API_KEY')
        print(f"GOOGLE_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    else:
        print("Generic provider - will test with LLM call only")
    
    # Universal LLM functionality test
    print(f"\n=== LLM Functionality Test ===")
    try:
        test_response = llm.call([{"role": "user", "content": "Respond with exactly: 'Test successful'"}])
        print("✅ LLM call successful!")
        print(f"Response type: {type(test_response)}")
        
        # Try to extract response content
        if hasattr(test_response, 'content'):
            print(f"Response content: {test_response.content[:100]}...")
        elif isinstance(test_response, str):
            print(f"Response: {test_response[:100]}...")
        else:
            print(f"Response: {str(test_response)[:100]}...")
            
    except Exception as e:
        print(f"❌ LLM call failed: {e}")
        print(f"Error type: {type(e).__name__}")
        
        # Provide specific troubleshooting tips based on error
        error_str = str(e).lower()
        if "connection" in error_str or "timeout" in error_str:
            print("💡 Tip: Check network connection and base_url configuration")
        elif "api_key" in error_str or "authentication" in error_str or "unauthorized" in error_str:
            print("💡 Tip: Check API key configuration and permissions")
        elif "model" in error_str or "not found" in error_str:
            print("💡 Tip: Verify model name and availability")

# Run the comprehensive test
test_llm_connection()

### Environment Variables Check for All LLM Providers

#### Environment Variables Audit Function

This function performs a comprehensive audit of API keys and environment variables for all major LLM providers:

- **Multi-Provider Support**: Checks for OpenAI, Anthropic, Google, Cohere, Hugging Face, Ollama, Azure, AWS, and more
- **Status Reporting**: Shows which API keys are configured and which are missing
- **Provider Discovery**: Identifies which LLM services you have access to
- **Security Check**: Verifies environment variable setup without exposing actual key values

📋 **Benefit**: Helps you understand your available LLM options and identify missing configurations.

In [ ]:
def check_environment_variables():
    """Check environment variables for all major LLM providers"""
    
    print("=== Environment Variables Status ===")
    
    # Common LLM provider environment variables
    env_vars = {
        "OpenAI": ["OPENAI_API_KEY", "OPENAI_BASE_URL"],
        "Anthropic": ["ANTHROPIC_API_KEY"],
        "Google": ["GOOGLE_API_KEY", "GOOGLE_APPLICATION_CREDENTIALS"],
        "Cohere": ["COHERE_API_KEY"],
        "Hugging Face": ["HUGGINGFACE_API_KEY", "HF_TOKEN"],
        "Ollama": ["OLLAMA_API_BASE", "OLLAMA_HOST"],
        "Azure OpenAI": ["AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT"],
        "AWS Bedrock": ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "AWS_REGION"],
        "Together AI": ["TOGETHER_API_KEY"],
        "Replicate": ["REPLICATE_API_TOKEN"],
        "Perplexity": ["PERPLEXITYAI_API_KEY"],
        "Groq": ["GROQ_API_KEY"]
    }
    
    found_providers = []
    
    for provider, vars_list in env_vars.items():
        provider_vars = {}
        has_any_var = False
        
        for var in vars_list:
            value = os.getenv(var)
            if value:
                provider_vars[var] = "✅ Set"
                has_any_var = True
            else:
                provider_vars[var] = "❌ Not set"
        
        if has_any_var:
            found_providers.append(provider)
            print(f"\n{provider}:")
            for var, status in provider_vars.items():
                print(f"  {var}: {status}")
    
    if not found_providers:
        print("No LLM provider environment variables found.")
        print("Make sure to set the appropriate API keys for your chosen provider.")
    else:
        print(f"\nConfigured providers: {', '.join(found_providers)}")

# Run environment check
check_environment_variables()

## Define the input fields the tool expects using Pydantic

### Define Input Schema with Pydantic

This code creates a structured input schema for our custom currency converter tool using Pydantic:

- **Type Safety**: Ensures `amount` is always a float and currencies are strings
- **Field Validation**: Pydantic automatically validates input types and required fields
- **Documentation**: Field descriptions help the AI agent understand what each parameter represents
- **Error Prevention**: Invalid inputs are caught before reaching the tool's execution logic

🎯 **Purpose**: Provides a clear contract for what data the tool expects, preventing runtime errors and improving AI agent reliability.

In [ ]:
class CurrencyConverterInput(BaseModel):
    """Input schema for CurrencyConverterTool."""
    amount: float = Field(..., description="The amount to convert.")
    from_currency: str = Field(..., description="The source currency code (e.g., 'USD').")
    to_currency: str = Field(..., description="The target currency code (e.g., 'EUR').")


## Define the Currency Converter Tool

### Custom Currency Converter Tool Implementation

This code defines a complete custom tool by extending CrewAI's `BaseTool` class. Here's how each component works:

#### **Class Attributes (Tool Metadata)**:
- **`name`**: Human-readable tool identifier shown to AI agents
- **`description`**: Explains the tool's purpose to help agents decide when to use it
- **`args_schema`**: Links to our Pydantic schema for input validation
- **`api_key`**: Securely retrieves the Exchange Rate API key from environment variables

#### **The `_run` Method - The Heart of the Tool** 🔧

The `_run` method is **absolutely critical** - it's the actual business logic that executes when an AI agent calls this tool:

1. **API Request**: Makes HTTP call to `exchangerate-api.com` with the source currency
2. **Error Handling**: Checks for API failures and returns helpful error messages
3. **Data Validation**: Verifies the target currency exists in the response
4. **Calculation**: Performs the currency conversion using real-time exchange rates
5. **Formatted Response**: Returns a human-readable result string

#### **Why `_run` is Essential**:
- **Execution Entry Point**: This is the ONLY method the CrewAI framework calls when an agent uses the tool
- **Type-Safe Inputs**: Parameters are automatically validated against our Pydantic schema before reaching `_run`
- **Return Format**: Must return a string that the AI agent can understand and use in its response
- **Error Resilience**: Should handle all possible failure scenarios gracefully

#### **Integration with CrewAI**:
When an AI agent needs currency conversion, CrewAI automatically:
1. Validates inputs against `CurrencyConverterInput` schema
2. Calls `_run` with the validated parameters
3. Provides the returned string to the agent for further processing

💡 **Key Insight**: The `_run` method transforms external API data into actionable information that AI agents can reason about and present to users.

In [ ]:
class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter Tool"
    description: str = "Converts an amount from one currency to another."
    args_schema: Type[BaseModel] = CurrencyConverterInput
    api_key: str = os.getenv("EXCHANGE_RATE_API_KEY")
    
    def _run(self, amount: float, from_currency: str, to_currency: str) -> str:
        url = f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/{from_currency}"
        response = requests.get(url)

        if response.status_code != 200:
            return "Failed to fetch exchange rates."

        data = response.json()
        if "conversion_rates" not in data or to_currency not in data["conversion_rates"]:
            return f"Invalid currency code: {to_currency}"

        rate = data["conversion_rates"][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equivalent to {converted_amount:.2f} {to_currency}."


## Define Currency Analyst Agent

### Create Specialized Currency Analyst Agent

This code defines an AI agent with domain expertise in currency analysis:

- **Role Definition**: Establishes the agent as a "Currency Analyst" with specific responsibilities
- **Goal Setting**: Defines the agent's primary objective (real-time conversions and financial insights)
- **Backstory**: Provides context and expertise that influences the agent's responses
- **Tool Integration**: Attaches our custom `CurrencyConverterTool` to give the agent access to real-time exchange rates
- **Verbose Mode**: Enables detailed logging to see the agent's reasoning process

🤖 **Agent Capabilities**: With the custom tool attached, this agent can now perform live currency conversions and provide context-aware financial advice.

In [ ]:
from crewai import Agent

currency_analyst = Agent(
    role="Currency Analyst",
    goal="Provide real-time currency conversions and financial insights.",
    backstory=(
        "You are a finance expert with deep knowledge of global exchange rates."
        "You help users with currency conversion and financial decision-making."
    ),
    tools=[CurrencyConverterTool()],  # Attach our custom tool
    verbose=True
)

## Define Currency Conversion Task

### Define Currency Conversion Task

This code creates a specific task that instructs the agent on how to use the currency converter tool:

- **Task Description**: Provides clear instructions with parameter placeholders (`{amount}`, `{from_currency}`, `{to_currency}`)
- **Expected Output**: Defines what kind of response format is desired from the agent
- **Agent Assignment**: Links this task to our specialized currency analyst agent
- **Dynamic Parameters**: Uses template variables that will be filled in when the crew executes

📝 **Task Flow**: The agent will receive this task, use the currency converter tool to get real-time rates, and provide both the conversion result and relevant financial context.

In [ ]:
from crewai import Task

currency_conversion_task = Task(
    description=(
        "Convert {amount} {from_currency} to {to_currency} "
        "using real-time exchange rates."
        "Provide the equivalent amount and "
        "explain any relevant financial context."
    ),
    expected_output=("A detailed response including the "
                     "converted amount and financial insights."),
    agent=currency_analyst
)

## Define Currency Conversion Crew

### Create and Execute Currency Conversion Crew

This final code block brings everything together and executes the currency conversion workflow:

#### **Crew Assembly**:
- **Agents List**: Includes our currency analyst agent with the attached custom tool
- **Tasks List**: Contains the currency conversion task with clear instructions
- **Process Type**: Sequential processing ensures tasks execute in order

#### **Crew Execution**:
- **`kickoff()`**: Starts the crew with specific input parameters (100 USD to EUR)
- **Input Mapping**: The template variables in the task description are filled with actual values
- **Tool Integration**: The agent automatically uses our custom currency converter tool to get real-time rates

#### **Response Handling**:
- **Raw Output**: Displays the agent's complete response including conversion results and financial insights
- **Markdown Rendering**: Formats the output nicely in the Jupyter notebook

🚀 **Complete Workflow**: This demonstrates the full pipeline from custom tool creation to agent execution with real-world API integration!

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[currency_analyst],
    tasks=[currency_conversion_task],
    process=Process.sequential
)

response = crew.kickoff(inputs={"amount": 100, 
                                "from_currency": "USD",
                                "to_currency": "EUR"})

# Print the response from the crew
print("=== Crew Response ===")
from IPython.display import Markdown
Markdown(response.raw)